In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import bsplines          as bsp
import Bspline           as bspline
import scipy.sparse      as sparse

#import STRUPHY_fields       as pic_fields
#import STRUPHY_pusher       as pic_pusher
import STRUPHY_accumulation as accumu

# 1. Test field evaluation

## 1. 1 B-field

In [9]:
Nel       = [8, 9, 10]                # mesh generation on logical domain
bc        = [False, False, False]     # boundary conditions (True: periodic, False: else)
p         = [3, 2, 3]                 # spline degrees  


el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
delta     = [1/Nel for Nel in Nel]                                             # element sizes
T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]

N         = [bspline.Bspline(T, p)     for T, p in zip(T, p)]
D         = [bspline.Bspline(t, p - 1) for t, p in zip(t, p)]


# ===================== coefficients for pp-forms in interval [0, delta] (N and D) ====================================
pp0 = []
pp1 = []

for i in range(3):
    if p[i] == 3:
        pp0.append(np.asfortranarray([[1/6, -1/(2*delta[i]), 1/(2*delta[i]**2), -1/(6*delta[i]**3)], [2/3, 0., -1/delta[i]**2, 1/(2*delta[i]**3)], [1/6, 1/(2*delta[i]), 1/(2*delta[i]**2), -1/(2*delta[i]**3)], [0., 0., 0., 1/(6*delta[i]**3)]]))
        pp1.append(np.asfortranarray([[1/2, -1/delta[i], 1/(2*delta[i]**2)], [1/2, 1/delta[i], -1/delta[i]**2], [0., 0., 1/(2*delta[i]**2)]])/delta[i])
    elif p[i] == 2:
        pp0.append(np.asfortranarray([[1/2, -1/delta[i], 1/(2*delta[i]**2)], [1/2, 1/delta[i], -1/delta[i]**2], [0., 0., 1/(2*delta[i]**2)]]))
        pp1.append(np.asfortranarray([[1., -1/delta[i]], [0., 1/delta[i]]])/delta[i])
    else:
        print('So far only cubic and quadratic splines implemented!')
# =====================================================================================================================


b1 = np.zeros((NbaseN[0], NbaseD[1], NbaseD[2]), order='F')
b2 = np.zeros((NbaseD[0], NbaseN[1], NbaseD[2]), order='F')
b3 = np.zeros((NbaseD[0], NbaseD[1], NbaseN[2]), order='F')

b1[:, :, :] = 0.1*(np.random.rand(NbaseN[0], NbaseD[1], NbaseD[2]) - 0.5)
b2[:, :, :] = 0.1*(np.random.rand(NbaseD[0], NbaseN[1], NbaseD[2]) - 0.5)
b3[:, :, :] = 0.1*(np.random.rand(NbaseD[0], NbaseD[1], NbaseN[2]) - 0.5)

Np        = int(20)
particles = np.asfortranarray(np.random.rand(Np, 7))
Nbase     = np.asfortranarray([[NbaseN[0], NbaseD[1], NbaseD[2]], [NbaseD[0], NbaseN[1], NbaseD[2]], [NbaseD[0], NbaseD[1], NbaseN[2]]])

In [10]:
B = np.empty((Np, 3), order='F')

for ip in range(Np):
    
    B[ip, 0] = 1.
    B[ip, 1] = 0.
    B[ip, 2] = 0.
    
    # evaluation of B1 - component (NDD)
    for i1 in range(NbaseN[0] + bc[0]*p[0]):
        for i2 in range(NbaseD[1] + bc[1]*(p[1] - 1)):
            for i3 in range(NbaseD[2] + bc[2]*(p[2] - 1)):

                N1 = N[0](particles[ip, 0], i1)
                D2 = D[1](particles[ip, 1], i2)*p[1]/(t[1][i2 + p[1]] - t[1][i2])
                D3 = D[2](particles[ip, 2], i3)*p[2]/(t[2][i3 + p[2]] - t[2][i3])

                B[ip, 0] += b1[i1%NbaseN[0], i2%NbaseD[1], i3%NbaseD[2]] * N1 * D2 * D3
                
    # evaluation of B2 - component (DND)
    for i1 in range(NbaseD[0] + bc[0]*(p[0] - 1)):
        for i2 in range(NbaseN[1] + bc[1]*p[1]):
            for i3 in range(NbaseD[2] + bc[2]*(p[2] - 1)):

                D1 = D[0](particles[ip, 0], i1)*p[0]/(t[0][i1 + p[0]] - t[0][i1])
                N2 = N[1](particles[ip, 1], i2)
                D3 = D[2](particles[ip, 2], i3)*p[2]/(t[2][i3 + p[2]] - t[2][i3])

                B[ip, 1] += b2[i1%NbaseD[0], i2%NbaseN[1], i3%NbaseD[2]] * D1 * N2 * D3
                
    # evaluation of B3 - component (DDN)
    for i1 in range(NbaseD[0] + bc[0]*(p[0] - 1)):
        for i2 in range(NbaseD[1] + bc[1]*(p[1] - 1)):
            for i3 in range(NbaseN[2] + bc[2]*p[2]):

                D1 = D[0](particles[ip, 0], i1)*p[0]/(t[0][i1 + p[0]] - t[0][i1])
                D2 = D[1](particles[ip, 1], i2)*p[1]/(t[1][i2 + p[1]] - t[1][i2])
                N3 = N[2](particles[ip, 2], i3)

                B[ip, 2] += b3[i1%NbaseD[0], i2%NbaseD[1], i3%NbaseN[2]] * D1 * D2 * N3

In [11]:
B_part    = np.empty((Np, 3), dtype=float, order='F')
pic_fields.evaluate_2form(particles[:, :3], T[0], T[1], T[2], t[0], t[1], t[2], p, Nel, Nbase, Np, b1, b2, b3, pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], B_part, 1, [1., 1., 1.])

In [12]:
np.allclose(B_part, B)

True

## 1. 2 U-field

In [6]:
Nel       = [8, 9, 10]                # mesh generation on logical domain
bc        = [False, False, False]     # boundary conditions (True: periodic, False: else)
p         = [3, 2, 3]                 # spline degrees  


el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
delta     = [1/Nel for Nel in Nel]                                             # element sizes
T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]

N         = [bspline.Bspline(T, p)     for T, p in zip(T, p)]
D         = [bspline.Bspline(t, p - 1) for t, p in zip(t, p)]


# ===================== coefficients for pp-forms in interval [0, delta] (N and D) ====================================
pp0 = []
pp1 = []

for i in range(3):
    if p[i] == 3:
        pp0.append(np.asfortranarray([[1/6, -1/(2*delta[i]), 1/(2*delta[i]**2), -1/(6*delta[i]**3)], [2/3, 0., -1/delta[i]**2, 1/(2*delta[i]**3)], [1/6, 1/(2*delta[i]), 1/(2*delta[i]**2), -1/(2*delta[i]**3)], [0., 0., 0., 1/(6*delta[i]**3)]]))
        pp1.append(np.asfortranarray([[1/2, -1/delta[i], 1/(2*delta[i]**2)], [1/2, 1/delta[i], -1/delta[i]**2], [0., 0., 1/(2*delta[i]**2)]])/delta[i])
    elif p[i] == 2:
        pp0.append(np.asfortranarray([[1/2, -1/delta[i], 1/(2*delta[i]**2)], [1/2, 1/delta[i], -1/delta[i]**2], [0., 0., 1/(2*delta[i]**2)]]))
        pp1.append(np.asfortranarray([[1., -1/delta[i]], [0., 1/delta[i]]])/delta[i])
    else:
        print('So far only cubic and quadratic splines implemented!')
# =====================================================================================================================


u1 = np.zeros((NbaseD[0], NbaseN[1], NbaseN[2]), order='F')
u2 = np.zeros((NbaseN[0], NbaseD[1], NbaseN[2]), order='F')
u3 = np.zeros((NbaseN[0], NbaseN[1], NbaseD[2]), order='F')

u1[:, :, :] = 0.1*(np.random.rand(NbaseD[0], NbaseN[1], NbaseN[2]) - 0.5)
u2[:, :, :] = 0.1*(np.random.rand(NbaseN[0], NbaseD[1], NbaseN[2]) - 0.5)
u3[:, :, :] = 0.1*(np.random.rand(NbaseN[0], NbaseN[1], NbaseD[2]) - 0.5)

Np        = int(20)
particles = np.asfortranarray(np.random.rand(Np, 7))
Nbase     = np.asfortranarray([[NbaseD[0], NbaseN[1], NbaseN[2]], [NbaseN[0], NbaseD[1], NbaseN[2]], [NbaseN[0], NbaseN[1], NbaseD[2]]])

In [7]:
U = np.empty((Np, 3), order='F')

for ip in range(Np):
    
    U[ip, 0] = 0.
    U[ip, 1] = 0.
    U[ip, 2] = 0.
    
    # evaluation of U1 - component (DNN)
    for i1 in range(NbaseD[0] + bc[0]*(p[0] - 1)):
        for i2 in range(NbaseN[1] + bc[1]*p[1]):
            for i3 in range(NbaseN[2] + bc[2]*p[2]):

                D1 = D[0](particles[ip, 0], i1)*p[0]/(t[0][i1 + p[0]] - t[0][i1])
                N2 = N[1](particles[ip, 1], i2)
                N3 = N[2](particles[ip, 2], i3)

                U[ip, 0] += u1[i1%NbaseD[0], i2%NbaseN[1], i3%NbaseN[2]] * D1 * N2 * N3
                
    # evaluation of U2 - component (NDN)
    for i1 in range(NbaseN[0] + bc[0]*p[0]):
        for i2 in range(NbaseD[1] + bc[1]*(p[1] - 1)):
            for i3 in range(NbaseN[2] + bc[2]*p[2]):

                N1 = N[0](particles[ip, 0], i1)
                D2 = D[1](particles[ip, 1], i2)*p[1]/(t[1][i2 + p[1]] - t[1][i2])
                N3 = N[2](particles[ip, 2], i3)

                U[ip, 1] += u2[i1%NbaseN[0], i2%NbaseD[1], i3%NbaseN[2]] * N1 * D2 * N3
                
    # evaluation of U3 - component (NND)
    for i1 in range(NbaseN[0] + bc[0]*p[0]):
        for i2 in range(NbaseN[1] + bc[1]*p[1]):
            for i3 in range(NbaseD[2] + bc[2]*(p[2] - 1)):

                N1 = N[0](particles[ip, 0], i1)
                N2 = N[1](particles[ip, 1], i2)
                D3 = D[2](particles[ip, 2], i3)*p[2]/(t[2][i3 + p[2]] - t[2][i3])

                U[ip, 2] += u3[i1%NbaseN[0], i2%NbaseN[1], i3%NbaseD[2]] * N1 * N2 * D3

In [8]:
U_part    = np.empty((Np, 3), dtype=float, order='F')
pic_fields.evaluate_1form(particles[:, :3], T[0], T[1], T[2], t[0], t[1], t[2], p, Nel, Nbase, Np, u1, u2, u3, pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], U_part, 1, [1., 1., 1.])

In [9]:
np.allclose(U_part, U)

True

# 2. Test pusher

## 2. 1 Step 3

In [2]:
L = [2., 1.5, 3.]

DF     = np.array([[  L[0], 0., 0.], [0.,   L[1], 0.], [0., 0.,   L[2]]])
DFinv  = np.array([[1/L[0], 0., 0.], [0., 1/L[1], 0.], [0., 0., 1/L[2]]])

G      = np.array([[  L[0]**2, 0., 0.], [0.,   L[1]**2, 0.], [0., 0.,   L[2]**2]])
Ginv   = np.array([[1/L[0]**2, 0., 0.], [0., 1/L[1]**2, 0.], [0., 0., 1/L[2]**2]])

g_sqrt = L[0]*L[1]*L[2]

Beq    = g_sqrt*DFinv.dot(np.array([1., 0., 0.]))
Ueq    = DF.T.dot(np.array([0., 0., 0.]))

Np        = 10
particles = np.asfortranarray(np.random.rand(Np, 7))

In [3]:
B        = np.asfortranarray(np.random.rand(Np, 3))
U        = np.asfortranarray(np.random.rand(Np, 3))

v_test   = np.zeros((Np, 3), dtype=float)

dt       = 0.15
B_prod   = np.zeros((3, 3), dtype=float)

for ip in range(Np):
                
    B_prod[0, 1] = -B[ip, 2]
    B_prod[0, 2] =  B[ip,1]
    
    B_prod[1, 0] =  B[ip,2]
    B_prod[1, 2] = -B[ip,0]
    
    B_prod[2, 0] = -B[ip,1]
    B_prod[2, 1] =  B[ip,0]
    
    temp = DFinv.T.dot(B_prod.dot(Ginv.dot(U[ip])))
    
    v_test[ip, :] = particles[ip, 3:6] + dt*temp

In [4]:
pic_pusher.pusher_step3(particles, dt, B, U, 1, L)

In [5]:
np.allclose(v_test, particles[:, 3:6])

True

## 2. 2 Step 5

In [3]:
L = [2., 1.5, 3.]

DF     = np.array([[  L[0], 0., 0.], [0.,   L[1], 0.], [0., 0.,   L[2]]])
DFinv  = np.array([[1/L[0], 0., 0.], [0., 1/L[1], 0.], [0., 0., 1/L[2]]])

G      = np.array([[  L[0]**2, 0., 0.], [0.,   L[1]**2, 0.], [0., 0.,   L[2]**2]])
Ginv   = np.array([[1/L[0]**2, 0., 0.], [0., 1/L[1]**2, 0.], [0., 0., 1/L[2]**2]])

g_sqrt = L[0]*L[1]*L[2]

Beq    = g_sqrt*DFinv.dot(np.array([1., 0., 0.]))
Ueq    = DF.T.dot(np.array([0., 0., 0.]))

Np        = 10
particles = np.asfortranarray(np.random.rand(Np, 7))

In [4]:
B        = np.asfortranarray(np.random.rand(Np, 3))

v_test   = np.zeros((Np, 3), dtype=float)

dt       = 0.15
B_prod   = np.zeros((3, 3), dtype=float)

for ip in range(Np):
                
    B_prod[0, 1] = -B[ip, 2]
    B_prod[0, 2] =  B[ip,1]
    
    B_prod[1, 0] =  B[ip,2]
    B_prod[1, 2] = -B[ip,0]
    
    B_prod[2, 0] = -B[ip,1]
    B_prod[2, 1] =  B[ip,0]
    
    mat = DFinv.T.dot(B_prod.dot(DFinv))
    
    lhs = np.identity(3) + dt/2*mat
    rhs = np.identity(3) - dt/2*mat
    
    v_test[ip, :] = np.linalg.solve(lhs, rhs.dot(particles[ip, 3:6]))

In [5]:
v_test

array([[0.84491426, 0.60515087, 0.82831242],
       [0.68920643, 0.62364991, 0.09546824],
       [0.38584562, 0.73207523, 0.86576213],
       [0.49834519, 0.4289784 , 0.85420407],
       [0.82296744, 0.06592607, 0.75101147],
       [0.84543623, 0.14380664, 0.33093046],
       [0.74222817, 0.33620382, 0.77661008],
       [0.3602305 , 0.25208931, 0.89070821],
       [0.75916295, 0.57627778, 0.7236691 ],
       [0.59986225, 0.16420252, 0.85877533]])

In [7]:
pic_pusher.pusher_step5(particles, dt, B, 1, L)

In [9]:
np.allclose(v_test, particles[:, 3:6])

True

# 3. Test accumulation

## 3. 1 Step 1

In [2]:
Nel       = [24, 8, 4]                # mesh generation on logical domain
bc        = [True, True, True]     # boundary conditions (True: periodic, False: else)
p         = [3, 3, 2]                 # spline degrees  


el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
delta     = [1/Nel for Nel in Nel]                                             # element sizes
T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]

N         = [bspline.Bspline(T, p)     for T, p in zip(T, p)]
D         = [bspline.Bspline(t, p - 1) for t, p in zip(t, p)]

L = [1., 1., 1.]

DF     = np.array([[  L[0], 0., 0.], [0.,   L[1], 0.], [0., 0.,   L[2]]])
DFinv  = np.array([[1/L[0], 0., 0.], [0., 1/L[1], 0.], [0., 0., 1/L[2]]])

G      = np.array([[  L[0]**2, 0., 0.], [0.,   L[1]**2, 0.], [0., 0.,   L[2]**2]])
Ginv   = np.array([[1/L[0]**2, 0., 0.], [0., 1/L[1]**2, 0.], [0., 0., 1/L[2]**2]])

g_sqrt = L[0]*L[1]*L[2]

Np        = int(2000)
particles = np.asfortranarray(np.random.rand(Np, 7))

Ntot = Nel[0]*Nel[1]*Nel[2]

In [3]:
B   = np.asfortranarray(np.random.rand(Np, 3))
acc = accumu.accumulation(T, p, bc)

In [4]:
#mat12,  mat13,  mat23  = acc.accumulation_step1(particles, B, 1, L)
mat12s, mat13s, mat23s = acc.accumulation_step1_sparse(particles, B, 1, L)

#mat12 = mat12.reshape(NbaseD[0]*NbaseN[1]*NbaseN[2], NbaseN[0]*NbaseD[1]*NbaseN[2])
#mat13 = mat13.reshape(NbaseD[0]*NbaseN[1]*NbaseN[2], NbaseN[0]*NbaseN[1]*NbaseD[2])
#mat23 = mat23.reshape(NbaseN[0]*NbaseD[1]*NbaseN[2], NbaseN[0]*NbaseN[1]*NbaseD[2])

mat12s = mat12s.toarray()
mat13s = mat13s.toarray()
mat23s = mat23s.toarray()

#mat12s = mat12s.reshape(Ntot, (2*p[0] + 1)*(2*p[1] + 1)*(2*p[2] + 1))
#mat13s = mat13s.reshape(Ntot, (2*p[0] + 1)*(2*p[1] + 1)*(2*p[2] + 1))
#mat23s = mat23s.reshape(Ntot, (2*p[0] + 1)*(2*p[1] + 1)*(2*p[2] + 1))

AttributeError: 'accumulation' object has no attribute 'accumulation_step1_sparse'

## 3. 2 Step 3

In [2]:
Nel       = [24, 8, 4]                # mesh generation on logical domain
bc        = [True, True, True]     # boundary conditions (True: periodic, False: else)
p         = [3, 3, 2]                 # spline degrees  


el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
delta     = [1/Nel for Nel in Nel]                                             # element sizes
T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]

N         = [bspline.Bspline(T, p)     for T, p in zip(T, p)]
D         = [bspline.Bspline(t, p - 1) for t, p in zip(t, p)]

L = [3., 1.5, 2.]

DF     = np.array([[  L[0], 0., 0.], [0.,   L[1], 0.], [0., 0.,   L[2]]])
DFinv  = np.array([[1/L[0], 0., 0.], [0., 1/L[1], 0.], [0., 0., 1/L[2]]])

G      = np.array([[  L[0]**2, 0., 0.], [0.,   L[1]**2, 0.], [0., 0.,   L[2]**2]])
Ginv   = np.array([[1/L[0]**2, 0., 0.], [0., 1/L[1]**2, 0.], [0., 0., 1/L[2]**2]])

g_sqrt = L[0]*L[1]*L[2]

Np        = int(20)
particles = np.asfortranarray(np.random.rand(Np, 7))

Ntot = Nel[0]*Nel[1]*Nel[2]

In [3]:
B      = np.asfortranarray(np.random.rand(Np, 3))
B_prod = np.zeros((3, 3), dtype=float)

mat1  = np.zeros((NbaseD[0], NbaseN[1], NbaseN[2], 3*Np), dtype=float, order='F')
mat2  = np.zeros((NbaseN[0], NbaseD[1], NbaseN[2], 3*Np), dtype=float, order='F')
mat3  = np.zeros((NbaseN[0], NbaseN[1], NbaseD[2], 3*Np), dtype=float, order='F')

In [4]:
for ip in range(Np):
    
    B_prod[0, 1] = -B[ip, 2]
    B_prod[0, 2] =  B[ip, 1]
    
    B_prod[1, 0] =  B[ip, 2]
    B_prod[1, 2] = -B[ip, 0]
    
    B_prod[2, 0] = -B[ip, 1]
    B_prod[2, 1] =  B[ip, 0]
    
    mat = Ginv.dot(B_prod.dot(DFinv))
    
    # evaluation of 1 - component (DNN)
    for i1 in range(NbaseD[0] + bc[0]*(p[0] - 1)):
        for i2 in range(NbaseN[1] + bc[1]*p[1]):
            for i3 in range(NbaseN[2] + bc[2]*p[2]):

                D1 = D[0](particles[ip, 0], i1)*p[0]/(t[0][i1 + p[0]] - t[0][i1])
                N2 = N[1](particles[ip, 1], i2)
                N3 = N[2](particles[ip, 2], i3)

                mat1[i1%NbaseD[0], i2%NbaseN[1], i3%NbaseN[2], ip + 0*Np] += D1 * N2 * N3 * mat[0, 0]
                mat1[i1%NbaseD[0], i2%NbaseN[1], i3%NbaseN[2], ip + 1*Np] += D1 * N2 * N3 * mat[0, 1]
                mat1[i1%NbaseD[0], i2%NbaseN[1], i3%NbaseN[2], ip + 2*Np] += D1 * N2 * N3 * mat[0, 2]
               
                
    # evaluation of 2 - component (NDN)
    for i1 in range(NbaseN[0] + bc[0]*p[0]):
        for i2 in range(NbaseD[1] + bc[1]*(p[1] - 1)):
            for i3 in range(NbaseN[2] + bc[2]*p[2]):
                
                N1 = N[0](particles[ip, 0], i1)
                D2 = D[1](particles[ip, 1], i2)*p[1]/(t[1][i2 + p[1]] - t[1][i2])
                N3 = N[2](particles[ip, 2], i3)

                mat2[i1%NbaseN[0], i2%NbaseD[1], i3%NbaseN[2], ip + 0*Np] += N1 * D2 * N3 * mat[1, 0]
                mat2[i1%NbaseN[0], i2%NbaseD[1], i3%NbaseN[2], ip + 1*Np] += N1 * D2 * N3 * mat[1, 1]
                mat2[i1%NbaseN[0], i2%NbaseD[1], i3%NbaseN[2], ip + 2*Np] += N1 * D2 * N3 * mat[1, 2]
                
                
    # evaluation of 3 - component (NND)
    for i1 in range(NbaseN[0] + bc[0]*p[0]):
        for i2 in range(NbaseN[1] + bc[1]*p[1]):
            for i3 in range(NbaseD[2] + bc[2]*(p[2] - 1)):
                
                N1 = N[0](particles[ip, 0], i1)
                N2 = N[1](particles[ip, 1], i2)
                D3 = D[2](particles[ip, 2], i3)*p[2]/(t[2][i3 + p[2]] - t[2][i3])

                mat3[i1%NbaseN[0], i2%NbaseN[1], i3%NbaseD[2], ip + 0*Np] += N1 * N2 * D3 * mat[2, 0]
                mat3[i1%NbaseN[0], i2%NbaseN[1], i3%NbaseD[2], ip + 1*Np] += N1 * N2 * D3 * mat[2, 1]
                mat3[i1%NbaseN[0], i2%NbaseN[1], i3%NbaseD[2], ip + 2*Np] += N1 * N2 * D3 * mat[2, 2]
                
                
J14 = np.vstack((mat1.reshape(NbaseD[0]*NbaseN[1]*NbaseN[2], 3*Np), mat2.reshape(NbaseN[0]*NbaseD[1]*NbaseN[2], 3*Np), mat3.reshape(NbaseN[0]*NbaseN[1]*NbaseD[2], 3*Np)))
W = np.diag(np.concatenate((particles[:, 6], particles[:, 6], particles[:, 6])))
V = np.concatenate((particles[:, 3], particles[:, 4], particles[:, 5]))
res = J14.dot(W).dot(V)

In [ ]:
acc = accumu.accumulation(T, p, bc)
Rs, vec = acc.accumulation_step3(particles, B, 1, L)

In [25]:
np.allclose(vec, res)

True

In [26]:
test = J14.dot(W).dot(J14.T)

In [27]:
np.allclose(test, Rs.toarray())

True

In [13]:
np.save('hello', Rs.toarray())

In [14]:
h = np.load('hello.npy')